In [1]:
import os
import datetime
import numpy as np
import pandas as pd
from scipy import spatial
import netCDF4 as nc
from matplotlib import pyplot as plt
from sklearn.metrics import roc_curve, auc
import read_data, concatenation, radar

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

In [55]:
radar_panel = \
read_data.radar_to_panel( '../data/data_20140811.nc',
               var_name='radar', time_unit='ns', dim_map = {'lat' : 'latitude', 'lon' :
                                             'longitude', 'time' : 'time'})

# Read vehicle data
veh = pd.read_csv('../data/camera_observations_1m.csv')
veh['Time'] = pd.to_datetime(veh['Time'])
veh.set_index('Time', inplace=True, drop=True)

# Correct vehicle data
ymin = radar_panel.major_axis.min()
ymax = radar_panel.major_axis.max()
xmin = radar_panel.minor_axis.min()
xmax = radar_panel.minor_axis.max()
veh = veh[(veh['Latitude'] > ymin) & (veh['Latitude'] < ymax) &
        (veh['Longitude'] > xmin) & (veh['Longitude'] < xmax)]
veh = veh[veh['GPS_Speed'] > 1]
invalid = [10150, 10151, 10589, 10139, 10615]
veh = veh[~veh['Device'].isin(invalid)]
veh['radar'] = veh['radar'].fillna(0)
veh = veh.dropna()
veh811 = veh.loc['2014-08-11']
veh612 = veh.loc['2014-06-12']
veh628 = veh.loc['2014-06-28']

# Read aggregated hi-resolution
comparison = pd.read_csv('../data/camera_observations_comparison.csv')
comparison['Time'] = pd.to_datetime(comparison['Time'])
comparison.set_index('Time', inplace=True)
comparison['radar'] = comparison['radar'].fillna(0)
comparison_sub = comparison.iloc[:, [0,1,2,3,4,5,-1,7,8]]
comparison_sub.columns = veh.columns
combined = pd.concat([comparison_sub, veh612, veh628])

# TPR and FPR when only the 1-minute resolution dataset is used

In [56]:
# When the wiper data is resampled to regular 1-minute increments, it sometimes "spills over"
# into neighboring intervals due to aggregation. We can use a threshold so that intervals containing very
# small fraction of wiper activity are not counted as raining
alpha = 0.05

In [57]:
perf = {'Wiper' : {}, 'Radar' : {}, 'Gage' : {}}

perf['Wiper']['TPR'] = ((veh['Wiper'] > alpha) & (veh['camera_rain'] != 0)).sum() / (veh['Wiper'] > alpha).sum()
perf['Wiper']['TNR'] = ((veh['Wiper'] <= alpha) & (veh['camera_rain'] == 0)).sum() / (veh['Wiper'] <= alpha).sum()
perf['Radar']['TPR'] = ((veh['radar'] > 0.0) & (veh['camera_rain'] != 0)).sum() / (veh['radar'] > 0.0).sum()
perf['Radar']['TNR'] = ((veh['radar'] == 0.0) & (veh['camera_rain'] == 0)).sum() / (veh['radar'] == 0.0).sum()
perf['Gage']['TPR'] = ((veh['gage'] > 0.0) & (veh['camera_rain'] != 0)).sum() / (veh['gage'] > 0.0).sum()
perf['Gage']['TNR'] = ((veh['gage'] == 0.0) & (veh['camera_rain'] == 0)).sum() / (veh['gage'] == 0.0).sum()
pd.DataFrame.from_dict(perf).sort_index(ascending=False)

,Gage,Radar,Wiper
TPR,0.817597,0.922892,0.959658
TNR,0.841614,0.965844,0.973177


In [58]:
# Not accounting for "spill over" (wipers still perform better than radar)
alpha = 0.0

In [59]:
perf = {'Wiper' : {}, 'Radar' : {}, 'Gage' : {}}

perf['Wiper']['TPR'] = ((veh['Wiper'] > alpha) & (veh['camera_rain'] != 0)).sum() / (veh['Wiper'] > alpha).sum()
perf['Wiper']['TNR'] = ((veh['Wiper'] <= alpha) & (veh['camera_rain'] == 0)).sum() / (veh['Wiper'] <= alpha).sum()
perf['Radar']['TPR'] = ((veh['radar'] > 0.0) & (veh['camera_rain'] != 0)).sum() / (veh['radar'] > 0.0).sum()
perf['Radar']['TNR'] = ((veh['radar'] == 0.0) & (veh['camera_rain'] == 0)).sum() / (veh['radar'] == 0.0).sum()
perf['Gage']['TPR'] = ((veh['gage'] > 0.0) & (veh['camera_rain'] != 0)).sum() / (veh['gage'] > 0.0).sum()
perf['Gage']['TNR'] = ((veh['gage'] == 0.0) & (veh['camera_rain'] == 0)).sum() / (veh['gage'] == 0.0).sum()
pd.DataFrame.from_dict(perf).sort_index(ascending=False)

,Gage,Radar,Wiper
TPR,0.817597,0.922892,0.945433
TNR,0.841614,0.965844,0.977498


# TPR and FPR when 2.4-second resolution ground-truth data are aggregated to 1-minute increments and used for 2014-08-11

#### Note that the 2.4-second resolution data is aggregated to 1-minute increments

In [60]:
alpha = 0.05

In [61]:
perf = {'Wiper' : {}, 'Radar' : {}, 'Gage' : {}}

perf['Wiper']['TPR'] = ((combined['Wiper'] > alpha) & (combined['camera_rain'] != 0)).sum() / (combined['Wiper'] > alpha).sum()
perf['Wiper']['TNR'] = ((combined['Wiper'] <= alpha) & (combined['camera_rain'] == 0)).sum() / (combined['Wiper'] <= alpha).sum()
perf['Radar']['TPR'] = ((combined['radar'] > 0.0) & (combined['camera_rain'] != 0)).sum() / (combined['radar'] > 0.0).sum()
perf['Radar']['TNR'] = ((combined['radar'] == 0.0) & (combined['camera_rain'] == 0)).sum() / (combined['radar'] == 0.0).sum()
perf['Gage']['TPR'] = ((combined['gage'] > 0.0) & (combined['camera_rain'] != 0)).sum() / (combined['gage'] > 0.0).sum()
perf['Gage']['TNR'] = ((combined['gage'] == 0.0) & (combined['camera_rain'] == 0)).sum() / (combined['gage'] == 0.0).sum()
pd.DataFrame.from_dict(perf).sort_index(ascending=False)

,Gage,Radar,Wiper
TPR,0.857143,0.934272,0.976716
TNR,0.823764,0.945772,0.946876


In [62]:
alpha = 0.0

In [63]:
perf = {'Wiper' : {}, 'Radar' : {}, 'Gage' : {}}

perf['Wiper']['TPR'] = ((combined['Wiper'] > alpha) & (combined['camera_rain'] != 0)).sum() / (combined['Wiper'] > alpha).sum()
perf['Wiper']['TNR'] = ((combined['Wiper'] <= alpha) & (combined['camera_rain'] == 0)).sum() / (combined['Wiper'] <= alpha).sum()
perf['Radar']['TPR'] = ((combined['radar'] > 0.0) & (combined['camera_rain'] != 0)).sum() / (combined['radar'] > 0.0).sum()
perf['Radar']['TNR'] = ((combined['radar'] == 0.0) & (combined['camera_rain'] == 0)).sum() / (combined['radar'] == 0.0).sum()
perf['Gage']['TPR'] = ((combined['gage'] > 0.0) & (combined['camera_rain'] != 0)).sum() / (combined['gage'] > 0.0).sum()
perf['Gage']['TNR'] = ((combined['gage'] == 0.0) & (combined['camera_rain'] == 0)).sum() / (combined['gage'] == 0.0).sum()
pd.DataFrame.from_dict(perf).sort_index(ascending=False)

,Gage,Radar,Wiper
TPR,0.857143,0.934272,0.966706
TNR,0.823764,0.945772,0.952417


# Cross comparison of camera observations

In [68]:
# True positive rate of low-resolution camera observations compared to aggregated hi-resolution ground truth
((hi_res['camera_lowres'] > 0.0) & (hi_res['camera_hires'] > 0.0)).sum() / (hi_res['camera_lowres'] > 0.0).sum()

0.9950310559006211

In [69]:
# True negative rate of low-resolution camera observations compared to aggregated hi-resolution ground truth
((hi_res['camera_lowres'] <= 0.0) & (hi_res['camera_hires'] <= 0.0)).sum() / (hi_res['camera_lowres'] <= 0.0).sum()

0.8931552587646077

In [70]:
# Fraction of time where both labelers detect rainfall vs. time where either labeler detects rainfall
((hi_res['camera_lowres'] > 0.0) & (hi_res['camera_hires'] > 0.0)).sum() / ((hi_res['camera_lowres'] > 0.0) | (hi_res['camera_hires'] > 0.0)).sum()

0.9217491369390104

In [71]:
# Fraction of time where neither labeler detects rainfall vs. time where either labeler detects rainfall
((hi_res['camera_lowres'] <= 0.0) & (hi_res['camera_hires'] <= 0.0)).sum() / ((hi_res['camera_lowres'] <= 0.0) | (hi_res['camera_hires'] <= 0.0)).sum()

0.8872305140961857